## ETL para analise de edições do fifa 22

In [49]:
import pandas as pd

basePath = '../tables/utf/'


#### Importa dados de jogadores

In [50]:
newPlayers = basePath + 'players.txt'

dfPlayers = pd.read_csv(newPlayers, sep='\s+')
dfPlayers = dfPlayers[['firstnameid', 'lastnameid', 'commonnameid', 'playerid']].drop_duplicates(subset='playerid', keep='first')

#### Trato dados de jogadores cruzando com bases de nomes

Busco nomes de base original da EA, e gero a combinação de nome completo

In [51]:
newPlayers = basePath + 'playernames.txt'

dfPlayersName = pd.read_csv(
    newPlayers,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8',  # Codificação UTF-8, equivalente a 65001
    names=['nameid', 'commentaryid', 'name'],  # Nomes das colunas
    skiprows=1,
    engine='python'  # Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='nameid', keep='first')

playersOriginalNames = dict(zip(dfPlayersName['nameid'], dfPlayersName['name']))

In [52]:
## First Name ID > 0 significa jogadores originais do fifa

dfOriginalPlayers = dfPlayers[dfPlayers['firstnameid'] > 0].copy()

colsToSearch = [('firstnameid','firstname'),('lastnameid', 'surname'), ('commonnameid','commonname')]

for col_id, col_name in colsToSearch:
    dfOriginalPlayers[col_name] = dfOriginalPlayers[col_id].map(playersOriginalNames)

dfOriginalPlayers['fullname'] = dfOriginalPlayers['firstname'] + ' ' + dfOriginalPlayers['surname']
dfOriginalPlayers.head()

,firstnameid,lastnameid,commonnameid,playerid,firstname,surname,commonname,fullname
1,15353,6531,0,27,Joe,Cole,NaN,Joe Cole
2,1882,14117,14116,41,Andrés,Iniesta Luján,Iniesta,Andrés Iniesta Luján
3,1178,29294,0,51,Alan,Shearer,NaN,Alan Shearer
4,27651,16635,0,240,Roy,Keane,NaN,Roy Keane
5,25044,28745,0,246,Paul,Scholes,NaN,Paul Scholes


Busco nome de jogadores editados 

In [53]:
editPlayersNames = basePath + 'editedplayernames.txt'
dfEditedPlayerNames = pd.read_csv(
    editPlayersNames,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8',  # Codificação UTF-8, equivalente a 65001
    names=['firstname','commonname','playerjerseyname','surname','playerid'],  # Nomes das colunas
    skiprows=1,
    engine='python'  # Usar o motor Python para maior flexibilidade
)

dfEditedPlayerNames.dropna(inplace=True)

In [54]:
## First Name ID = 0 significa jogadores editados do mod
dfEditedPlayers = dfPlayers[dfPlayers['firstnameid'] == 0]

dfEditedPlayers= dfEditedPlayers.merge(dfEditedPlayerNames[['playerid','firstname','surname','commonname']], on='playerid', how='left')
dfEditedPlayers['fullname'] = dfEditedPlayers['firstname'] + ' ' + dfEditedPlayers['surname']
dfEditedPlayerNames.head()

,firstname,commonname,playerjerseyname,surname,playerid
0,Yago Rafael,Yago Darub,Yago Darub,Valadares Darub,22
1,Paulo Vítor,Paulo Vítor,Paulo Vítor,Leal Sousa Lima,3000
2,Miguel Angel,Miguelito,Miguelito,Terceros Acuna,4330
3,Luiz Carlos,Mirandinha,Mirandinha,Paulino de Carvalho,4331
4,Luiz Henrique,Luiz Henrique,Luiz Henrique,Bezerra dos Santos,4332


Mesclo bases de nomes editados e originais, transformando em base unica.

In [55]:
dfPlayers = pd.concat([dfOriginalPlayers,dfEditedPlayers])
dfPlayers.drop(columns=['firstnameid','lastnameid','commonnameid'],inplace=True)

### Inicia busca de times

In [56]:
teamsPath = basePath + 'teams.txt'

dfTeams = pd.read_csv(
    teamsPath,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='teamid')
dfTeams = dfTeams[['teamid', 'teamname']].dropna()

Retira seleções da listagem de times.

In [57]:
## Busco base de ligas e a base relacional de ligas e times

leaguesPath = basePath + 'leagues.txt'
leaguesTeamsPath = basePath + 'leagueteamlinks.txt'

dfLeagues = pd.read_csv(
    leaguesPath,
    usecols=['leagueid', 'leaguename'],
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
).drop_duplicates(subset='leagueid')

dfLeaguesTeams = pd.read_csv(
    leaguesTeamsPath,
    usecols=['leagueid', 'teamid', 'artificialkey'],
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
)

## Retiro a seleção por base do id das ligas de seleção masculina e feminina
condition = (dfLeaguesTeams['leagueid'] != 2136) & (dfLeaguesTeams['leagueid'] != 78 ) & (dfLeaguesTeams['artificialkey'] != 0) & (dfLeaguesTeams['teamid'] != 112190) & (dfLeaguesTeams['teamid'] != 111596) & (dfLeaguesTeams['teamid'] != 112428)
dfLeaguesTeams = dfLeaguesTeams.loc[condition]

dfLeaguesTeams = dfLeaguesTeams.merge(dfLeagues, on='leagueid', how='left')
# dfLeaguesTeams.drop_duplicates(subset='teamid')
dfLeaguesTeams.head()


,leagueid,artificialkey,teamid,leaguename
0,1,1,270,3F Superliga
1,1,2,271,3F Superliga
2,1,3,272,3F Superliga
3,1,4,819,3F Superliga
4,1,5,820,3F Superliga


In [58]:
dfteamsFull = dfTeams.merge(dfLeaguesTeams, on='teamid', how="inner")

Importo a tablea relacional entre times e jogadores

In [59]:
teamsPlayersPath = basePath + 'teamplayerlinks.txt'

dfTeamsPlayers = pd.read_csv(
    teamsPlayersPath,
    sep='\t',  # Delimitador de tabulação
    encoding='utf-8'# Usar o motor Python para maior flexibilidade
)
dfTeamsPlayers = dfTeamsPlayers[['teamid','playerid','jerseynumber', 'position']].dropna()

dfTeamsPlayers.head()

,teamid,playerid,jerseynumber,position
0,1,270390,21,29
1,1,199503,34,11
2,1,201118,17,3
3,1,209989,5,9
4,1,213051,25,28


In [60]:
dfNewTeamsPlayers = dfTeamsPlayers.merge(dfteamsFull, on='teamid', how='right')
dfNewTeamsPlayers = dfNewTeamsPlayers.merge(dfPlayers,on='playerid', how='left')
dfNewTeamsPlayers = dfNewTeamsPlayers.query('playerid > 0')
# dfNewTeamsPlayers = dfNewTeamsPlayers.merge(dfTeams,on='teamid', how='left')
dfNewTeamsPlayers.to_csv('./result/oldPlayers.csv')
# subColumn = ['playerid']
# has_duplicates = dfNewTeamsPlayers[dfNewTeamsPlayers.duplicated(subset=subColumn,keep=False)]
# has_duplicates = has_duplicates.sort_values(by=['playerid'])
# has_duplicates.to_csv('./result/duplicates.csv')
# print(has_duplicates.head())